# Glofas Forecast

In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
import climetlab as cml
from climetlab_cems_flood.adapter import Adapter
import climetlab_cems_flood as cmf

In [3]:
nwse = [50.972204,5.450796, 46.296530, 11.871059] # Ryne

In [12]:

class MyMerger():
    def __init__(self, *args, **kwargs):
        pass
    def merge(self, paths, **kwargs):
        return xr.open_mfdataset(paths)


In [21]:
forecast1 = cml.load_dataset(
            'cems-flood-glofas-forecast',
            model='lisflood',
            product_type='ensemble_perturbed_forecasts',
            system_version='operational',
            period= '20210710-20210715',#'2001-200401-04*',
            leadtime_hour = '24-120',
            variable="river_discharge_in_the_last_24_hours",
            area= nwse,
            split_on = ['day'],
            threads = 6,
            merger = "merge"
        )

In [22]:
forecast1 

Home page,-
Documentation,-
Citation,-
Licence,-


In [23]:
forecast1.to_xarray()

ValueError: Could not find any dimension coordinates to use to order the datasets for concatenation

In [24]:
ada = Adapter(forecast1, 
              output_folder= '/home/iacopo/temp', 
              output_name_prefix= 'test_fc')

In [25]:
ada.to_zarr(True, 
            {"dis24":{"realization":10,
                      "forecast_reference_time": 1,
                      "leadtime": -1,
                      "lat": 100,
                      "lon": 100},
            "forecast_reference_time":None,
            "lat":None,
            "lon":None,
            "leadtime":None},
            '25M', 
            'temp.zarr')

ValueError: Could not find any dimension coordinates to use to order the datasets for concatenation

In [ ]:
ada.to_netcdf()

In [ ]:
24*5

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.figure(figsize=(15,10))
for i,c in enumerate(["black","grey","blue","red","green","orange"]):
    _ = plt.plot(ds.dis24.isel(forecast_reference_time=i).time.values,
                 ds.dis24.isel(forecast_reference_time=i,drop=True).mean(["lat","lon"]).values.T,
                 color=c)

In [ ]:
forecast = cml.load_dataset(
            'cems-flood-glofas-forecast',
            model='lisflood',
            product_type='ensemble_perturbed_forecasts',
            system_version='operational',
            period= '20210710-20210715',#'2001-200401-04*',
            leadtime_hour = '24-720',
            variable="river_discharge_in_the_last_24_hours",
            area= nwse, # list of ROI
            split_on = [('day',1)],
            threads = 5
            # output_folder = //
            # output_format = 'zarr'
            # output_name = "{dataset}_{year}_{month}_{day}_{area}"
            # merger
            # filter
        )

In [ ]:
from itertools import chain, product
from copy import deepcopy

In [ ]:
req = { "year":["2000","2001","2002","2003","2004","2005"], "month":["01","02","03","04","05","06","07"],"var":["soil", "rain"]}

In [ ]:
split = ["year","month", "var"]

In [ ]:
[i for i in list(range(10))]

In [ ]:
split = [("year",2),("month",3), "var"]
split_mod = [i if isinstance(i, tuple) else (i,None) for i in split]
split_mod

In [ ]:
def chunked(out_list, chunk_size):
    if chunk_size is None:
        return out_list
    return [out_list[i:i+chunk_size] for i in range(0, len(out_list), chunk_size)]

In [ ]:
[chunked(req[s[0]],s[1]) for s in split_mod]

In [ ]:
list(product(*[chunked(req[s[0]],s[1]) for s in split_mod] ))

In [ ]:
for n in list(product(*[chunked(req[s[0]],s[1]) for s in split_mod] )):
    d = { k[0]:v for k,v in zip(split_mod,n)}
    r = deepcopy(req)
    r.update(d)
    print(r)